In [1]:
import pandas as pd
import numpy as np
from math import floor
import math
import os
from datetime import datetime
import xarray as xr

# Listado de playas con avistamientos y fechas

In [2]:
avistamientos_df = pd.read_excel("./Physalia_Ambiental_R.xlsx")
# avistamientos_df = pd.read_excel("../Physala_Data/Datos_Physalia_20171010.xls")

columnas = avistamientos_df.iloc[0]

#Quito las 3 primeras filas, debido al formato de la excell
avistamientos_df = avistamientos_df.iloc[3:] 
avistamientos_df.columns = columnas

# Me quedo solo con los datos de avistamientos
avistamientos_df = avistamientos_df[["Latitud","Longitud","Año","Mes","Día","Avistamientos"]]

#Transdormaciones para sacar con fecha (datetime)
avistamientos_fecha_df=avistamientos_df[["Año","Mes","Día"]]
avistamientos_fecha_df.columns = ["year","month","day"]

fecha = pd.to_datetime(avistamientos_fecha_df)

avistamientos_df["Fecha"]=fecha
avistamientos_df=avistamientos_df[["Latitud","Longitud","Fecha","Avistamientos"]]
avistamientos_df.head()

,Latitud,Longitud,Fecha,Avistamientos
3,-25.4,-70.4833,2014-05-01,1
4,-18.4631,-70.3053,2014-05-01,1
5,-27.0658,-70.8259,2014-05-09,13
6,-18.4933,-70.3261,2014-05-10,15
7,-27.122,-70.8576,2014-05-10,23


# Exporto listado de playas sin redondear coordenadas

In [3]:
# Genera excel con las coordenadas de las playas
playas_df = avistamientos_df.reset_index()
playas_df = playas_df[['Latitud','Longitud']]
playas_df.to_excel('./playasSinRedondeoCoord.xlsx')

In [4]:
base = 1/4
def redondeo(x,y):
    return  [base * round(x/base),base * round(y/base)]
x = redondeo(37.1707222222222,-73.2053333333333)
x

[37.25, -73.25]

# Redondeo a cada cuarto de grado
Redondeo de latitud y longitud para juntar lecturas de una misma playa con coordenadas muy proximas

Se exporta el dataframe generado a un excel

In [5]:
base=1/4
def floorQuarter(x):
    return (floor(x * 4) / 4.0) 

def redondeo(x):
    return  base * round(x/base)

#Se añaden atributos con la longitud y latitud redondeadas a cada cuarto de grado
avistamientos_df["Lat_floor"] = avistamientos_df.Latitud.map(redondeo)
avistamientos_df["Long_floor"] = avistamientos_df.Longitud.map(redondeo)

df = avistamientos_df[['Lat_floor', 'Long_floor',"Fecha","Avistamientos"]]
df = df.set_index(['Lat_floor', 'Long_floor',"Fecha"])

#DataFrame total avistamientos de playas en esa cuadricula
df_sum = df.groupby(['Lat_floor', 'Long_floor',"Fecha"]).sum()

#DataFrame número de playas en esa cuadricula
df_count = df.groupby(['Lat_floor', 'Long_floor',"Fecha"]).count()

#Dataframe con el total de avistamientos y el número de playas
df_join = df_sum.join(df_count,lsuffix="I",rsuffix="R")
df_join.columns=['Suma', 'N_Playas']
df_join['Media_Playas'] = df_join.Suma /df_join.N_Playas

# DataFrame de avistamientos procesado
df_join.to_excel("AvistamientosConRedondeoCoord-025.xlsx")

df_join.head()

Suma  N_Playas  Media_Playas
Lat_floor Long_floor Fecha                                   
-42.75    -74.00     2015-05-27     1         1           1.0
-41.75    -73.75     2014-06-04     1         1           1.0
                     2015-06-10    20         1          20.0
                     2015-06-11   151         1         151.0
          -73.50     2015-05-23     1         1           1.0

# Redondeo con salto entre cuadrantes de dataSet original (0,0833)
Redondeo de latitud y longitud para juntar lecturas de una misma playa con coordenadas muy proximas

Se exporta el dataframe generado a un excel

In [6]:
base=1/12
def floorTwelfth(x):
    return (floor(x * 12) / 12.0)

def redondeo(x):
    return  base * round(x/base)

#Se añaden atributos con la longitud y latitud redondeadas a cada cuarto de grado
avistamientos_df["Lat_floor"] = avistamientos_df.Latitud.map(redondeo)
avistamientos_df["Long_floor"] = avistamientos_df.Longitud.map(redondeo)

df = avistamientos_df[['Lat_floor', 'Long_floor',"Fecha","Avistamientos"]]
df = df.set_index(['Lat_floor', 'Long_floor',"Fecha"])

#DataFrame total avistamientos de playas en esa cuadricula
df_sum = df.groupby(['Lat_floor', 'Long_floor',"Fecha"]).sum()

#DataFrame número de playas en esa cuadricula
df_count = df.groupby(['Lat_floor', 'Long_floor',"Fecha"]).count()

#Dataframe con el total de avistamientos y el número de playas
df_join = df_sum.join(df_count,lsuffix="I",rsuffix="R")
df_join.columns=['Suma', 'N_Playas']
df_join['Media_Playas'] = df_join.Suma /df_join.N_Playas

# DataFrame de avistamientos procesado
df_join.to_excel("AvistamientosConRedondeoCoord-00833.xlsx")

In [7]:
df_join.head()

Suma  N_Playas  Media_Playas
Lat_floor  Long_floor Fecha                                   
-42.666667 -74.083333 2015-05-27     1         1           1.0
-41.833333 -73.833333 2014-06-04     1         1           1.0
           -73.500000 2015-05-23     1         1           1.0
-41.666667 -73.750000 2015-06-10    20         1          20.0
                      2015-06-11   151         1         151.0

## Exporto listado de coordenadas para utilixarlo en la web

In [8]:
df_web = df_join.reset_index().rename(columns={"Lat_floor": "Latitud", "Long_floor": "Longitud"})[['Latitud','Longitud']]
df_web.to_excel("Playas.xlsx")

# Estructura Buena

In [9]:
# DF con playas
# df_playas = pd.read_excel('./AvistamientosConRedondeoCoord-025.xlsx').reset_index()
df_playas = pd.read_excel('./AvistamientosConRedondeoCoord-00833.xlsx').reset_index()
'''
Las celdas que estaban unidas por tener el mismo valor, tienen valores nulos
por lo que las relleno con us valor correspodiente
'''
df_playas = df_playas.fillna(method= 'ffill',axis=0)
# Cojo las columnas que me interesan
df_playas = df_playas[["Suma","Lat_floor","Long_floor","Fecha"]]
# Renombro las columnas 
df_playas = df_playas.rename(columns={"Suma": "Avistamientos", "Lat_floor": "Latitud", "Long_floor": "Longitud"})
# Ordeno por fecha
df_playas = df_playas.sort_values(by=["Fecha"])

In [10]:
df_playas # DataFrame con avistamientos en cada playa
cuadrantes = { # Numero de cuadrantes que se concatenarán con las playas
    'izquierda': 3 ,
    'arriba': 0,
    'abajo': 0
}
salto = 1/12 # Salto de coordenadas entre los cuadrantes
listado_archivos = os.listdir('../../descargas') # Listo todos los archivos de Copernicus
variables = ['Coord','Profundidad','mlotst','zos','bottomT','thetao','so','uo','vo']

In [32]:
def busca_archivo(fecha):
    texto ='_{}_'.format(str(fecha).split()[0].replace('-',''))
    archivo = [x for x in listado_archivos if str(texto) in x]
    data = xr.open_dataset('../../descargas/{}'.format(archivo[0])) # cargo el archivo
    return data # devuelvo dataset

def dame_datos(latitud,longitud,ds):
    return ds.sel({'latitude':latitud,'longitude': longitud})

def comprueba_datos(latitud,longitud,ds):
    valor = dame_datos(latitud,longitud,ds)
    while math.isnan(valor.mlotst[0]):
        longitud = longitud - salto
        valor = dame_datos(latitud,longitud,ds)
    return latitud,longitud # devuelvo las coordenadas con datos

def crea_DF():
    df_resultados = None
    df_resultados = pd.DataFrame(columns=["Avistamientos","Latitud","Longitud","Fecha",'Coord',"Profundidad","mlotst",'zos','bottomT','thetao','so','uo','vo']) # columnas básicas
    for izquierda in range(cuadrantes['izquierda']): # por cada cuadrante a la izquierda meto un grupo de columnas
        var_aux = [str(x+'_izq_'+str(izquierda+1)) for x in variables]
        df_aux = pd.DataFrame(columns=var_aux)
        df_resultados = df_resultados.join(df_aux)
        for superior in range (cuadrantes['arriba']): # por cada cuadrante a la superior meto un grupo de columnas
            var_aux = [x+'_sup_'+str(izquierda+1)+'.'+str(superior+1) for x in variables]
            df_aux = pd.DataFrame(columns=var_aux)
            df_resultados = df_resultados.join(df_aux)
        for inferior in range (cuadrantes['abajo']): # por cada cuadrante a la inferior meto un grupo de columnas    
            var_aux = [x+'_inf_'+str(izquierda+1)+'.'+str(inferior+1) for x in variables]
            df_aux = pd.DataFrame(columns=var_aux)
            df_resultados = df_resultados.join(df_aux)
    return df_resultados

def meto_en_df(df_resultados,row,coordenadas,df,indice,variables_actuales):
    for x in range(3):
        df_resultados.loc[indice + x,["Avistamientos","Latitud","Longitud","Fecha"]] = row.values.tolist() # meto en df_resulados los datos de ese cuadrante
        vars = df.loc[x,['depth', 'mlotst','zos','bottomT','thetao','so','uo','vo']].values.tolist()
        vars.insert(0,coordenadas)
        df_resultados.loc[indice + x,variables_actuales] = vars # meto coordenadas reales utilizadas y datos 
    return df_resultados

def dameCoordIzq(lat,long,n_saltos):
    return [lat,long-(salto*n_saltos)]

def dameCoordSuperior(lat,long,n_saltos):
    coordenadas = dameCoordIzq(lat,long,n_saltos)
    return [lat+(salto*n_saltos),long]

def dameCoordInferior(lat,long,n_saltos):
    return [lat-(salto*n_saltos),long]

In [33]:
def crea_estr():
    df_resultados = crea_DF() # creo df con columnas necesarias
    #print(df_resultados)
    for contador,[index, row] in enumerate(df_playas.iterrows()): # por cada playa
        print(contador,end='\r')
        ds = busca_archivo(row['Fecha']) # busco el archivo de la fecha 
        coordenadas = comprueba_datos(row['Latitud'],row['Longitud'],ds) # miro a ver si el cuadrante tiene datos, sino cojo el siguiente por la izquierda

        df = dame_datos(coordenadas[0],coordenadas[1],ds).to_dataframe().reset_index() # dataframe con los datso oceanicos de esas coordenadas y en esa fecha
        df_resultados = meto_en_df(df_resultados,row,list(coordenadas),df,contador * 3,variables)

        # datos de las celdas extras
        for izquierda in range(cuadrantes['izquierda']): # por cada cuadrante a la izquierda, añado datos
            coordenadas = dameCoordIzq(coordenadas[0],coordenadas[1],izquierda+1)
            df = dame_datos(coordenadas[0],coordenadas[1],ds).to_dataframe().reset_index() 
            var_aux = [x+'_izq_'+str(izquierda+1) for x in variables]
            df_resultados = meto_en_df(df_resultados,row,coordenadas,df,contador * 3,var_aux)
            coord_aux = coordenadas
            for superior in range (cuadrantes['arriba']): # por cada cuadrante a la superior, añado datos
                coord_aux = dameCoordSuperior(coord_aux[0],coord_aux[1],superior+1)
                df = dame_datos(coord_aux[0],coord_aux[1],ds).to_dataframe().reset_index() 
                var_aux = [x+'_sup_'+str(izquierda+1)+'.'+str(superior+1) for x in variables]
                df_resultados = meto_en_df(df_resultados,row,coord_aux,df,contador * 3,var_aux)
            coord_aux = coordenadas
            for inferior in range (cuadrantes['abajo']): # por cada cuadrante a la inferior, añado datos  
                coord_aux = dameCoordInferior(coord_aux[0],coord_aux[1],inferior+1)
                df = dame_datos(coord_aux[0],coord_aux[1],ds).to_dataframe().reset_index() 
                var_aux = [x+'_inf_'+str(izquierda+1)+'.'+str(inferior+1) for x in variables]
                df_resultados = meto_en_df(df_resultados,row,coord_aux,df,contador * 3,var_aux)

#         if contador == 5:
#             break
    return df_resultados

In [34]:
df_resultados = crea_estr()
df_resultados

,Avistamientos,Latitud,Longitud,Fecha,Coord,Profundidad,mlotst,zos,bottomT,thetao,...,vo_izq_2,Coord_izq_3,Profundidad_izq_3,mlotst_izq_3,zos_izq_3,bottomT_izq_3,thetao_izq_3,so_izq_3,uo_izq_3,vo_izq_3
0,1,-25.4167,-70.5,2014-05-01 00:00:00,"[-25.41666666666666, -70.58333333333333]",0.494025,12.5126,0.194708,12.5373,15.8861,...,-0.151372,"[-25.41666666666666, -71.08333333333333]",0.494025,15.8696,0.138249,1.3932,17.5597,34.8186,-0.14771,-0.0561541
1,1,-25.4167,-70.5,2014-05-01 00:00:00,"[-25.41666666666666, -70.58333333333333]",5.07822,12.5126,0.194708,12.5373,15.8524,...,-0.178228,"[-25.41666666666666, -71.08333333333333]",5.07822,15.8696,0.138249,1.3932,17.5487,34.8186,-0.151982,-0.0848415
2,1,-25.4167,-70.5,2014-05-01 00:00:00,"[-25.41666666666666, -70.58333333333333]",9.573,12.5126,0.194708,12.5373,15.8209,...,-0.188604,"[-25.41666666666666, -71.08333333333333]",9.573,15.8696,0.138249,1.3932,17.5341,34.8186,-0.151982,-0.0970489
3,1,-18.5,-70.3333,2014-05-01 00:00:00,"[-18.5, -70.33333333333333]",0.494025,10.5289,0.219428,15.3323,17.6198,...,-0.0311289,"[-18.5, -70.83333333333333]",0.494025,12.8178,0.209357,4.47533,21.8262,35.2428,0.121464,-0.16419
4,1,-18.5,-70.3333,2014-05-01 00:00:00,"[-18.5, -70.33333333333333]",5.07822,10.5289,0.219428,15.3323,17.2711,...,-0.0598163,"[-18.5, -70.83333333333333]",5.07822,12.8178,0.209357,4.47533,21.8152,35.2412,0.127567,-0.190435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1564,2,-26.3333,-70.6667,2016-03-13 00:00:00,"[-26.33333333333333, -70.66666666666666]",5.07822,10.6815,0.0592059,13.9825,18.7726,...,0.375988,"[-26.33333333333333, -71.16666666666666]",5.07822,11.9022,0.137638,1.54848,21.9881,34.8064,-0.0555437,0.0616474
1565,2,-26.3333,-70.6667,2016-03-13 00:00:00,"[-26.33333333333333, -70.66666666666666]",9.573,10.6815,0.0592059,13.9825,17.4872,...,0.341197,"[-26.33333333333333, -71.16666666666666]",9.573,11.9022,0.137638,1.54848,21.9434,34.8033,-0.0561541,0.0476089
1566,5,-27,-70.75,2016-03-13 00:00:00,"[-27.0, -70.83333333333333]",0.494025,10.5289,0.0723289,13.1306,19.8596,...,0.0860622,"[-27.0, -71.33333333333333]",0.494025,10.5289,0.108951,1.36244,20.4829,34.6904,0.120853,0.191046
1567,5,-27,-70.75,2016-03-13 00:00:00,"[-27.0, -70.83333333333333]",5.07822,10.5289,0.0723289,13.1306,19.6274,...,0.0592059,"[-27.0, -71.33333333333333]",5.07822,10.5289,0.108951,1.36244,20.354,34.6889,0.11475,0.162358


In [14]:
df_resultados.to_excel('EstructuraFinal.xlsx')


## Estructura sin rellenar huecos sin datos

In [15]:
def crea_estr():
    df_resultados = crea_DF() # creo df con columnas necesarias
    #print(df_resultados)
    for contador,[index, row] in enumerate(df_playas.iterrows()): # por cada playa
        print(contador,end='\r')
        ds = busca_archivo(row['Fecha']) # busco el archivo de la fecha 
        coordenadas = [row['Latitud'],row['Longitud']]

        df = dame_datos(row['Latitud'],row['Longitud'],ds).to_dataframe().reset_index() # dataframe con los datso oceanicos de esas coordenadas y en esa fecha
        df_resultados = meto_en_df(df_resultados,row,[row['Latitud'],row['Longitud']],df,contador * 3,variables)
        
        # datos de las celdas extras
        for izquierda in range(cuadrantes['izquierda']): # por cada cuadrante a la izquierda, añado datos
            coordenadas = dameCoordIzq(coordenadas[0],coordenadas[1],izquierda+1)
            df = dame_datos(coordenadas[0],coordenadas[1],ds).to_dataframe().reset_index() 
            var_aux = [x+'_izq_'+str(izquierda+1) for x in variables]
            df_resultados = meto_en_df(df_resultados,row,coordenadas,df,contador * 3,var_aux)
            coord_aux = coordenadas
            for superior in range (cuadrantes['arriba']): # por cada cuadrante a la superior, añado datos
                coord_aux = dameCoordSuperior(coord_aux[0],coord_aux[1],superior+1)
                df = dame_datos(coord_aux[0],coord_aux[1],ds).to_dataframe().reset_index() 
                var_aux = [x+'_sup_'+str(izquierda+1)+'.'+str(superior+1) for x in variables]
                df_resultados = meto_en_df(df_resultados,row,coord_aux,df,contador * 3,var_aux)
            coord_aux = coordenadas
            for inferior in range (cuadrantes['abajo']): # por cada cuadrante a la inferior, añado datos  
                coord_aux = dameCoordInferior(coord_aux[0],coord_aux[1],inferior+1)
                df = dame_datos(coord_aux[0],coord_aux[1],ds).to_dataframe().reset_index() 
                var_aux = [x+'_inf_'+str(izquierda+1)+'.'+str(inferior+1) for x in variables]
                df_resultados = meto_en_df(df_resultados,row,coord_aux,df,contador * 3,var_aux)
        
    return df_resultados

In [16]:
df_resultados_con_vacios = crea_estr()

In [17]:
df_resultados_con_vacios.head()

,Avistamientos,Latitud,Longitud,Fecha,Coord,Profundidad,mlotst,zos,bottomT,thetao,...,vo_izq_2,Coord_izq_3,Profundidad_izq_3,mlotst_izq_3,zos_izq_3,bottomT_izq_3,thetao_izq_3,so_izq_3,uo_izq_3,vo_izq_3
0,1,-25.4167,-70.5,2014-05-01 00:00:00,"[-25.41666666666666, -70.5]",0.494025,NaN,NaN,NaN,NaN,...,-0.237434,"[-25.41666666666666, -71.0]",0.494025,15.4118,0.142216,1.48183,17.1649,34.8079,-0.130009,-0.0195318
1,1,-25.4167,-70.5,2014-05-01 00:00:00,"[-25.41666666666666, -70.5]",5.07822,NaN,NaN,NaN,NaN,...,-0.26368,"[-25.41666666666666, -71.0]",5.07822,15.4118,0.142216,1.48183,17.1481,34.8079,-0.135502,-0.0482192
2,1,-25.4167,-70.5,2014-05-01 00:00:00,"[-25.41666666666666, -70.5]",9.573,NaN,NaN,NaN,NaN,...,-0.273446,"[-25.41666666666666, -71.0]",9.573,15.4118,0.142216,1.48183,17.1312,34.8079,-0.136723,-0.0616474
3,1,-18.5,-70.3333,2014-05-01 00:00:00,"[-18.5, -70.33333333333333]",0.494025,10.5289,0.219428,15.3323,17.6198,...,-0.0311289,"[-18.5, -70.83333333333333]",0.494025,12.8178,0.209357,4.47533,21.8262,35.2428,0.121464,-0.16419
4,1,-18.5,-70.3333,2014-05-01 00:00:00,"[-18.5, -70.33333333333333]",5.07822,10.5289,0.219428,15.3323,17.2711,...,-0.0598163,"[-18.5, -70.83333333333333]",5.07822,12.8178,0.209357,4.47533,21.8152,35.2412,0.127567,-0.190435


In [18]:
df_resultados_con_vacios.to_excel('EstructuraConVacios.xlsx')

# Rellenar datos de celdas vacios
Con datos de la coordenada más cercana por la izquierda

In [46]:
def comprueba_datos_thetao(latitud,longitud,ds):
    valor = dame_datos(latitud,longitud,ds)
    while math.isnan(valor.thetao[0][2]):
        longitud = float(longitud) - salto
        valor = dame_datos(latitud,longitud,ds)
    return latitud,longitud # devuelvo las coordenadas con datos

In [36]:
datos_excel = pd.read_excel('EstructuraFinal.xlsx') # leo estructura de datos que contiene huecos sin informacion
datos = datos_excel.drop(['Unnamed: 0'],axis=1).sort_values(by=['Latitud','Longitud','Profundidad','Fecha']) 

In [41]:
datos.head()

,Avistamientos,Latitud,Longitud,Fecha,Coord,Profundidad,mlotst,zos,bottomT,thetao,...,vo_izq_2,Coord_izq_3,Profundidad_izq_3,mlotst_izq_3,zos_izq_3,bottomT_izq_3,thetao_izq_3,so_izq_3,uo_izq_3,vo_izq_3
381,1,-42.666667,-74.083333,2015-05-27,"[-42.66666666666666, -74.16666666666666]",0.494025,49.592583,0.106815,12.775384,12.781243,...,-0.057985,"[-42.66666666666666, -74.66666666666666]",0.494025,48.219250,0.108951,9.026734,13.219977,33.463547,0.120853,0.030519
382,1,-42.666667,-74.083333,2015-05-27,"[-42.66666666666666, -74.16666666666666]",5.078224,49.592583,0.106815,12.775384,12.780511,...,-0.030519,"[-42.66666666666666, -74.66666666666666]",5.078224,48.219250,0.108951,9.026734,13.220710,33.463547,0.114139,0.057375
383,1,-42.666667,-74.083333,2015-05-27,"[-42.66666666666666, -74.16666666666666]",9.572997,49.592583,0.106815,12.775384,12.780511,...,-0.017701,"[-42.66666666666666, -74.66666666666666]",9.572997,48.219250,0.108951,9.026734,13.220710,33.463547,0.108036,0.071413
21,1,-41.833333,-73.833333,2014-06-04,"[-41.83333333333333, -73.83333333333333]",0.494025,11.139256,0.058901,11.645955,11.601275,...,-0.161138,"[-41.83333333333333, -74.33333333333333]",0.494025,15.259255,0.012513,10.200842,11.333934,33.634449,-0.029908,-0.153813
22,1,-41.833333,-73.833333,2014-06-04,"[-41.83333333333333, -73.83333333333333]",5.078224,11.139256,0.058901,11.645955,11.628376,...,-0.163579,"[-41.83333333333333, -74.33333333333333]",5.078224,15.259255,0.012513,10.200842,11.333934,33.645130,-0.031739,-0.153203


In [51]:
def rellenar_huecos(df):
    filas_nan = df[df.isna().any(axis=1)] # filas con valores vacios
    datos_copia = df.copy()
    for indice, fila in filas_nan.iterrows():
    #     print(indice,fila)
        elem = fila.isna() # columnas con valores nan
        for index, value in elem.items():
            if value:
                nombre_original= index
                nombre = 'Coord'
                try:
                    nombre_original = index[:index.index('_')] # nombre de la columna en el DataSet
                    nombre = nombre + index[index.index('_'):] # nombre de la columna de coordenadas de ese punto
                except:
                    pass
                coordenadas = fila[nombre].replace('[','').replace(']','').replace(' ','').split(',')
                archivo = busca_archivo(fila['Fecha'])
                coordenadas = comprueba_datos_thetao(coordenadas[0],coordenadas[1],archivo)
                datos_fila = dame_datos(coordenadas[0],coordenadas[1],archivo)
                valor =datos_fila.sel({'depth':fila['Profundidad']}).to_dataframe().reset_index()[nombre_original].values[0]
                datos_copia.loc[indice,index] = valor
    return datos_copia

In [52]:
datos_sin_huecos = rellenar_huecos(datos)

In [53]:
filas_nan_2 = datos_sin_huecos[datos_sin_huecos.isna().any(axis=1)] # compruebo que se hayan rellenado todos los vacios
filas_nan_2.head()

,Avistamientos,Latitud,Longitud,Fecha,Coord,Profundidad,mlotst,zos,bottomT,thetao,...,vo_izq_2,Coord_izq_3,Profundidad_izq_3,mlotst_izq_3,zos_izq_3,bottomT_izq_3,thetao_izq_3,so_izq_3,uo_izq_3,vo_izq_3


In [54]:
datos_sin_huecos.to_excel('EstructuraFinalSinVacios.xlsx')